In [ ]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, expr, split
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException
from pyspark.sql.types import StructType, StructField, StringType, BinaryType, IntegerType, DoubleType, TimestampType, DateType, LongType
from delta.tables import DeltaTable
from pyspark.sql.utils import AnalysisException
from pyspark.storagelevel import StorageLevel
from typing import Union, Optional
from pyspark.sql.functions import input_file_name

# --- Credenciais AWS ---
accessKeyId = ""
secretAccessKey = ""

# --- Sessão Spark ---
def create_spark_session() -> SparkSession:
    spark = (
        SparkSession
        .builder
        .appName("Silver Zone Streaming")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .enableHiveSupport()
        .getOrCreate()
    )
    
    spark.sparkContext.setLogLevel("WARN")

    conf = spark.sparkContext._jsc.hadoopConfiguration()
    conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider")
    conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    conf.set("fs.s3a.fast.upload", "true")
    conf.set("fs.s3a.bucket.all.committer.magic.enabled", "true")
    conf.set("fs.s3a.directory.marker.retention", "keep")
    conf.set("spark.driver.extraClassPath", "/usr/local/spark/jars/*")
    conf.set("spark.driver.memory", "8g")
    conf.set("spark.executor.memory", "16g")
    conf.set("fs.s3a.access.key", accessKeyId)
    conf.set("fs.s3a.secret.key", secretAccessKey)

    return spark

spark = create_spark_session()

In [39]:
# Paths
bronze_path = "s3a://dev-lab-02-us-east-2-bronze/spotify/"
silver_path = "s3a://dev-lab-02-us-east-2-silver/"
silver_table = "fato_streamings"
silver_table_path = f"{silver_path}{silver_table}"
checkpoint_path = f"{silver_path}/checkpoints/{silver_table}"

In [48]:
usuarios_df = spark.readStream \
    .format("delta") \
    .option("ignoreChanges", "true") \
    .load(f"{bronze_path}usuarios")

streamings_df = spark.readStream \
    .format("delta") \
    .option("ignoreChanges", "true") \
    .load(f"{bronze_path}streamings")

In [49]:
df_result = (
    streamings_df.alias("s")
    .join(usuarios_df.alias("u"), col("s.nome") == col("u.nome"), "inner")
    .withColumn("masked_email", expr("""
        CONCAT(
            SUBSTRING(u.email, 1, 1),
            REPEAT('*', INSTR(u.email, '@') - 2),
            SUBSTRING(u.email, INSTR(u.email, '@'), LENGTH(u.email))
        )
    """))
    .withColumn("musica_bruta", col("s.musica"))
    .withColumn("artista", split(col("musica_bruta"), "-")[0])
    .withColumn("musica", split(col("musica_bruta"), "-")[1])
    .withColumn("flg_feat", expr("INSTR(split(musica_bruta, '-')[1], 'w/') > 0"))
    .select(
        col("s.id"),
        col("u.id").alias("id_usuario"),
        col("masked_email"),
        col("artista"),
        col("musica"),
        col("flg_feat"),
        col("s.timestamp"),
        col("s.origem_arquivo")
    )
)

In [50]:
def upsert_to_delta(microbatch_df, batch_id):
    if microbatch_df.rdd.isEmpty():
        return

    windowed_df = (
        microbatch_df
        .withColumn("timestamp", F.col("timestamp").cast("timestamp"))
        .dropDuplicates(["id"])  # garante que não haja duplicatas por ID no microbatch
    )
    
    if DeltaTable.isDeltaTable(spark, silver_table_path):
        delta_table = DeltaTable.forPath(spark, silver_table_path)

        delta_table.alias("target").merge(
            windowed_df.alias("source"),
            "target.id = source.id"
        ).whenMatchedUpdateAll(
        ).whenNotMatchedInsertAll(
        ).execute()

    else:
        windowed_df.write.format("delta").mode("overwrite").save(silver_table_path)

In [ ]:
query = (
    df_result.writeStream
    .foreachBatch(upsert_to_delta)
    .outputMode("append")
    .trigger(processingTime="1 minute")
    .option("checkpointLocation", f"{checkpoint_path}")
    .start()
)

query.awaitTermination()

## Modos de saída do Spark Streaming (outputMode)

No Spark Streaming, o parâmetro 'outputMode' define como os dados processados nas janelas de tempo (batches)
devem ser gravados na saída, seja para um arquivo, banco de dados ou outro destino.
Existem três modos principais:

#### 1. 'append' ➕
 - Apenas os novos dados (que chegaram no batch atual) são adicionados à saída.
 - Ideal para cenários onde você só precisa de registros novos, sem reprocessar os antigos.
 - É o modo padrão 
 
Exemplo:

Em um stream de vendas, você apenas adiciona novas vendas ao seu arquivo ou banco de dados, 
sem alterar os registros anteriores.

```python
append_example = """
# Exemplo de código para 'append' ➕
streaming_data \
  .writeStream \
  .outputMode("append") \
  .format("parquet") \
  .option("checkpointLocation", "/path/to/checkpoint") \
  .start("/path/to/output")
"""

```
#### 2. 'complete' 🔄
- Todos os dados processados até o momento são recalculados e escritos na saída.
- Ideal para cenários onde você precisa de um estado completo de todos os dados processados até aquele ponto,
  como somar todos os valores ou gerar uma visão agregada.

Exemplo:

Em um stream de contagem de palavras, você recalcula e escreve a contagem total de todas as palavras 
desde o início, não apenas as novas.
```python
complete_example = """
# Exemplo de código para 'complete' 🔄
streaming_data \
  .groupBy("word") \
  .count() \
  .writeStream \
  .outputMode("complete") \
  .format("parquet") \
  .option("checkpointLocation", "/path/to/checkpoint") \
  .start("/path/to/output")
"""
```
#### 3. 'update' 🔄✏️
- Apenas os dados que foram atualizados (ou mudaram) desde a última vez que foram processados são gravados na saída.
- Ideal para cenários onde você não precisa recalcular tudo, mas deseja registrar as atualizações.

Exemplo:

Em um stream de contagem de palavras, você apenas atualiza a contagem das palavras que mudaram, 
sem recalcular todas as contagens.

```python
update_example = """
# Exemplo de código para 'update' 🔄✏️
streaming_data \
  .groupBy("word") \
  .count() \
  .writeStream \
  .outputMode("update") \
  .format("parquet") \
  .option("checkpointLocation", "/path/to/checkpoint") \
  .start("/path/to/output")
"""
```
#### Resumo:
- **append** ➕: apenas os novos dados são gravados.
- **complete** 🔄: todos os dados são recalculados e gravados.
- **update** 🔄✏️: apenas as atualizações dos dados são gravadas.

> Escolher o 'outputMode' adequado depende do tipo de processamento e dos requisitos da aplicação.
```
